In [1]:
import pandas as pd
import numpy as np
import math

# Load the datasets.

In [2]:
ratings = pd.read_csv("C:\\Users\\fishk\\Documents\\School Work\Year 4\\Semester One\\IS Project II\\Test Datasets\\ratings.csv")
movies = pd.read_csv("C:\\Users\\fishk\\Documents\\School Work\Year 4\\Semester One\\IS Project II\\Test Datasets\\movies.csv")
tags = pd.read_csv("C:\\Users\\fishk\\Documents\\School Work\Year 4\\Semester One\\IS Project II\\Test Datasets\\tags.csv")

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


# Compute the item TF-IDF values

In [6]:
#builds a TF dataframe
tf = tags.groupby(['movieId', 'tag'], as_index=False, sort=False).count().rename(columns = {"userId" : 'tag_count_tf'})[['movieId', 'tag', 'tag_count_tf']]
tag_distinct = tags[['tag', 'movieId']].drop_duplicates()

In [10]:
#builds a IDF dataframe
df = tag_distinct.groupby(['tag'], as_index = False, sort = False).count().rename(columns = {'movieId': 'tag_count_DF'})[['tag','tag_count_DF']]

In [11]:
#compute TF-IDF values
idf = math.log10(len(np.unique(tags['movieId'])))
df['idf'] = idf-np.log10(df['tag_count_DF'])

tf = pd.merge(tf, df, on='tag', how='left', sort = False)
tf['tf-idf'] = tf['tag_count_tf'] * tf['idf']

#show TF-IDF values for each movie
tf[['movieId', 'tag', 'tf-idf']].head()

,movieId,tag,tf-idf
0,60756,funny,5.622700
1,60756,Highly quotable,2.719331
2,60756,will ferrell,8.686268
3,89774,Boxing story,3.196453
4,89774,MMA,3.196453


# Calculate unit length vector of TF

In [12]:
#get the vector length with rows of movieId and columns of TF-IDF
vector_length = tf.loc[:,('movieId','tf-idf')]

In [13]:
# normalize the vector by unit length
vector_length['tf-idf-sq'] = vector_length['tf-idf']**2
vector_length = vector_length.groupby(['movieId'], as_index=False, sort=False).sum().rename(columns = {'tf-idf-sq':'tf-idf-sq-total'})[['movieId','tf-idf-sq-total']]
vector_length['vector_length'] = np.sqrt(vector_length[['tf-idf-sq-total']].sum(axis=1))


In [14]:
tf = pd.merge(tf, vector_length, on='movieId', how='left', sort=False)
tf['tag_vector'] = tf['tf-idf']/tf['vector_length']

In [15]:
tf[['movieId','tag','tf-idf','vector_length','tag_vector']].head()

,movieId,tag,tf-idf,vector_length,tag_vector
0,60756,funny,5.622700,10.887728,0.516425
1,60756,Highly quotable,2.719331,10.887728,0.249761
2,60756,will ferrell,8.686268,10.887728,0.797804
3,89774,Boxing story,3.196453,5.368248,0.595437
4,89774,MMA,3.196453,5.368248,0.595437


# Compute the user profile vector

In [16]:
ratings_filter = ratings[ratings['rating']>=3]
user_distinct = np.unique(ratings['userId'])
user_tag_pref = pd.DataFrame()
i = 1

In [20]:
# enter userId for analysis
userId = 1

# compute the profile vector for the selected user
user_index = user_distinct.tolist().index(userId)

for user in user_distinct[user_index:user_index+1]:
    
    if i%30==0:
        print ("user: ", i , "out of: ", len(user_distinct))
            
    user_data= ratings_filter[ratings_filter['userId']==user]
    user_data = pd.merge(tf,user_data, on = 'movieId', how = 'inner', sort = False)
    user_data_itr = user_data.groupby(['tag'], as_index = False, sort = False).sum().rename(columns = {'tag_vector': 'tag_pref'})[['tag','tag_pref']]
    user_data_itr['user']=user
    user_tag_pref = user_tag_pref.append(user_data_itr, ignore_index=True)
    i=i+1

#display the user profile vector: 'tag_pref'
user_tag_pref.head()


,tag,tag_pref,user
0,beautiful scenery,0.305940,1
1,epic,0.305940,1
2,historical,0.337748,1
3,inspirational,0.263892,1
4,Medieval,0.337748,1



# Compute cosine similarities between the two vectors i.e. user profile and movies

In [ ]:
#other methods to be considered: linear_kernel()

In [22]:
#compute the proximity of movies to a particular user profile
user_distinct = np.unique(ratings_filter['userId'])
tag_merge_all = pd.DataFrame()
i = 1

# enter userId for analysis
userId = 1

# compute the cosine similarity
user_index = user_distinct.tolist().index(userId)

for user in user_distinct[user_index:user_index+1]:

    user_tag_pref_all = user_tag_pref[user_tag_pref['user']==user]
    movie_distinct = np.unique(tf['movieId'])
    j = 1
    
for movie in movie_distinct:
        
        if j%1000==0:
            print ("movie: ", j, "out of: ", len(movie_distinct), "with user: ", i, "out of: ", len(user_distinct))
        
        tf_movie = tf[tf['movieId']==movie]
        tag_merge = pd.merge(tf_movie, user_tag_pref_all, on = 'tag', how = 'left', sort = False)
        tag_merge['tag_pref'] = tag_merge['tag_pref'].fillna(0)
        tag_merge['tag_value'] = tag_merge['tag_vector']*tag_merge['tag_pref']
        
        tag_vec_val = np.sqrt(np.sum(np.square(tag_merge['tag_vector']), axis=0))
        tag_pref_val = np.sqrt(np.sum(np.square(user_tag_pref_all['tag_pref']), axis=0))
        
        tag_merge_final = tag_merge.groupby(['user','movieId'])[['tag_value']].sum().rename(columns = {'tag_value': 'rating'}).reset_index()
        
        tag_merge_final['rating']=tag_merge_final['rating']/(tag_vec_val*tag_pref_val)
        
        tag_merge_all = tag_merge_all.append(tag_merge_final, ignore_index=True)
        j=j+1
    
        i=i+1
    
tag_merge_all = tag_merge_all.sort_values(['user','rating'], ascending=False)

# remove movies already rated by user
movies_rated = ratings[ratings['userId'] == userId]['movieId']
tag_merge_all = tag_merge_all[~tag_merge_all['movieId'].isin(movies_rated)]
tag_merge_all['user'] = tag_merge_all['user'].apply(np.int64)

movie:  1000 out of:  1572 with user:  1000 out of:  609


In [23]:
# display the top 10 movies by rating
tag_merge_all.head(10)

,user,movieId,rating
71,1,588.0,0.491913
76,1,594.0,0.491913
77,1,595.0,0.491913
80,1,616.0,0.491913
116,1,1022.0,0.491913
122,1,1033.0,0.491913
254,1,2080.0,0.491913
255,1,2085.0,0.491913
115,1,1010.0,0.370264
598,1,8961.0,0.366724
